# Downloading necessary programs

In [1]:
#installing necessary programs
!pip install bayesian-optimization
!pip install feature_engine
!pip install catboost
!pip install scikit-optimize 
!pip install matplotlib
!pip install category_encoders

You should consider upgrading via the 'c:\users\klee9\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\klee9\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\klee9\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\klee9\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\klee9\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\klee9\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [3]:
from __future__ import division, print_function, unicode_literals
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import scipy as sp
from scipy import stats
import os
import itertools
from IPython.display import Image
from sklearn import tree
from sklearn.datasets import load_digits
from sklearn.tree import export_graphviz
%matplotlib inline
pd.set_option('display.max_columns', 30)
from  warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import confusion_matrix, classification_report, plot_roc_curve
from datetime import timedelta
from sklearn.utils import resample
from sklearn.model_selection import KFold
from sklearn import neighbors, datasets
from sklearn.model_selection import train_test_split, KFold, cross_val_score,learning_curve, ShuffleSplit, StratifiedKFold, validation_curve, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, f1_score, cohen_kappa_score,  confusion_matrix, classification_report, auc,roc_curve, log_loss, roc_auc_score
from numpy import mean
from numpy import std
import random
import catboost as cb
from bayes_opt import BayesianOptimization
from skopt.space import Real, Integer
from catboost import Pool
from time import time
import pprint
import category_encoders as ce
import warnings
warnings.filterwarnings("ignore")

# Data cleaning and feature engineering 

In [4]:
#downloading the data and randomly sampling 15 million rows 
df=pd.read_csv(r"C:\Users\klee9\Downloads\ProjectTrainingData.csv")
data=df.sample(n=15000000, random_state=23)

In [5]:
#selecting the rest of the data not in the training set as validation data
val_data=df.drop(data.index)

In [9]:
#validation feature engineering and cleaning
val_data['hr'] = val_data['hour'].astype(str).str[-2:]
val_data['day'] = val_data['hour'].astype(str).str[4:6]
val_data['C1'] = val_data['C1'].astype(str)
val_data['banner_pos'] = val_data['banner_pos'].astype(str)
val_data['device_type'] = val_data['device_type'].astype(str)
val_data['device_conn_type'] = val_data['device_conn_type'].astype(str)
val_data['C14'] = val_data['C14'].astype(str)
val_data['C15'] = val_data['C15'].astype(str)
val_data['C16'] = val_data['C16'].astype(str)
val_data['C17'] = val_data['C17'].astype(str)
val_data['C18'] = val_data['C18'].astype(str)
val_data['C19'] = val_data['C19'].astype(str)
val_data['C20'] = val_data['C20'].astype(str)
val_data['C21'] = val_data['C21'].astype(str)
val_data['click'] = val_data['click'].astype(str)
val_data = val_data.drop('hour', 1)

#validation cardinality reduction
val_data.loc[val_data['site_id'].value_counts()[val_data['site_id']].values < (.000001*val_data.shape[0]), 'site_id'] = "Other"
val_data.loc[val_data['site_domain'].value_counts()[val_data['site_domain']].values < (.000001*val_data.shape[0]), 'site_domain'] = "Other"
val_data.loc[val_data['app_id'].value_counts()[val_data['app_id']].values < (.000001*val_data.shape[0]), 'app_id'] = "Other"
val_data.loc[val_data['app_domain'].value_counts()[val_data['app_domain']].values < (.000001*val_data.shape[0]), 'app_domain'] = "Other"
val_data.loc[val_data['device_id'].value_counts()[val_data['device_id']].values < (.000001*val_data.shape[0]), 'device_id'] = "Other"
val_data.loc[val_data['device_ip'].value_counts()[val_data['device_ip']].values < (.000001*val_data.shape[0]), 'device_ip'] = "Other"
val_data.loc[val_data['device_model'].value_counts()[val_data['device_model']].values < (.000001*val_data.shape[0]), 'device_model'] = "Other"
val_data.loc[val_data['C14'].value_counts()[val_data['C14']].values < (.000001*val_data.shape[0]), 'C14'] = "Other"
val_data.loc[val_data['C17'].value_counts()[val_data['C17']].values < (.000001*val_data.shape[0]), 'C17'] = "Other"
val_data.loc[val_data['C20'].value_counts()[val_data['C20']].values < (.000001*val_data.shape[0]), 'C20'] = "Other"

val_x=val_data.drop(["click", "id"], axis=1)
val_y=val_data["click"]

val_x=val_x.values
val_x=val_x[:,2:]
val_y=val_y.values

In [5]:
# training data formatting and feature engineering
data['hr'] = data['hour'].astype(str).str[-2:]
data['day'] = data['hour'].astype(str).str[4:6]
data['C1'] = data['C1'].astype(str)
data['banner_pos'] = data['banner_pos'].astype(str)
data['device_type'] = data['device_type'].astype(str)
data['device_conn_type'] = data['device_conn_type'].astype(str)
data['C14'] = data['C14'].astype(str)
data['C15'] = data['C15'].astype(str)
data['C16'] = data['C16'].astype(str)
data['C17'] = data['C17'].astype(str)
data['C18'] = data['C18'].astype(str)
data['C19'] = data['C19'].astype(str)
data['C20'] = data['C20'].astype(str)
data['C21'] = data['C21'].astype(str)
data['click'] = data['click'].astype(str)
data = data.drop('hour', 1)

#cardinality reduction
data.loc[data['site_id'].value_counts()[data['site_id']].values < (.000001*data.shape[0]), 'site_id'] = "Other"
data.loc[data['site_domain'].value_counts()[data['site_domain']].values < (.000001*data.shape[0]), 'site_domain'] = "Other"
data.loc[data['app_id'].value_counts()[data['app_id']].values < (.000001*data.shape[0]), 'app_id'] = "Other"
data.loc[data['app_domain'].value_counts()[data['app_domain']].values < (.000001*data.shape[0]), 'app_domain'] = "Other"
data.loc[data['device_id'].value_counts()[data['device_id']].values < (.000001*data.shape[0]), 'device_id'] = "Other"
data.loc[data['device_ip'].value_counts()[data['device_ip']].values < (.000001*data.shape[0]), 'device_ip'] = "Other"
data.loc[data['device_model'].value_counts()[data['device_model']].values < (.000001*data.shape[0]), 'device_model'] = "Other"
data.loc[data['C14'].value_counts()[data['C14']].values < (.000001*data.shape[0]), 'C14'] = "Other"
data.loc[data['C17'].value_counts()[data['C17']].values < (.000001*data.shape[0]), 'C17'] = "Other"
data.loc[data['C20'].value_counts()[data['C20']].values < (.000001*data.shape[0]), 'C20'] = "Other"

# Model building and training

In [8]:
#setting up the x and y variables
X=data.drop(["click", "id"], axis=1)
y=data["click"]

#splititng to a train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=89)

#converting the training and test set into numpy arrays for faster calculations
X_train=X_train.values
X_test=X_test.values
X_train = X_train[:,2:]
X_test = X_test[:,2:]

y_train = y_train.values
y_test = y_test.values

#labeling the categorical columns to feed into Catboost
cat_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]


#setting uo the Catboost classifier
catmodel = cb.CatBoostClassifier(
    loss_function='Logloss',
    custom_metric='Logloss',
    random_seed=123,
    verbose=1,
    thread_count=-1,
    od_type='Iter',
    od_wait=10,
    iterations=500,
    cat_features=cat_features)

#fitting the model
catboost=catmodel.fit(
    X_train, y_train,
eval_set=[(X_test, y_test)],
plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.435928
0:	learn: 0.4809872	test: 0.4805502	best: 0.4805502 (0)	total: 33.2s	remaining: 4h 36m 10s
1:	learn: 0.4244257	test: 0.4237871	best: 0.4237871 (1)	total: 52.4s	remaining: 3h 37m 38s
2:	learn: 0.4100202	test: 0.4094452	best: 0.4094452 (2)	total: 1m 18s	remaining: 3h 37m 6s
3:	learn: 0.4052197	test: 0.4046129	best: 0.4046129 (3)	total: 1m 42s	remaining: 3h 30m 48s
4:	learn: 0.4020459	test: 0.4013946	best: 0.4013946 (4)	total: 2m 13s	remaining: 3h 40m 19s
5:	learn: 0.3999859	test: 0.3991821	best: 0.3991821 (5)	total: 2m 40s	remaining: 3h 40m 24s
6:	learn: 0.3980368	test: 0.3969889	best: 0.3969889 (6)	total: 3m 8s	remaining: 3h 41m 42s
7:	learn: 0.3966831	test: 0.3951451	best: 0.3951451 (7)	total: 3m 29s	remaining: 3h 35m 5s
8:	learn: 0.3956249	test: 0.3939886	best: 0.3939886 (8)	total: 3m 59s	remaining: 3h 38m 6s
9:	learn: 0.3951672	test: 0.3934275	best: 0.3934275 (9)	total: 4m 22s	remaining: 3h 34m 35s
10:	learn: 0.3939708	test: 0.3915392	best: 0.3915392 (10

88:	learn: 0.3866912	test: 0.3823944	best: 0.3823944 (88)	total: 37m 30s	remaining: 2h 53m 14s
89:	learn: 0.3866712	test: 0.3823720	best: 0.3823720 (89)	total: 38m	remaining: 2h 53m 9s
90:	learn: 0.3866507	test: 0.3823566	best: 0.3823566 (90)	total: 38m 27s	remaining: 2h 52m 52s
91:	learn: 0.3866299	test: 0.3823412	best: 0.3823412 (91)	total: 38m 54s	remaining: 2h 52m 32s
92:	learn: 0.3866152	test: 0.3823298	best: 0.3823298 (92)	total: 39m 13s	remaining: 2h 51m 38s
93:	learn: 0.3866082	test: 0.3823248	best: 0.3823248 (93)	total: 39m 38s	remaining: 2h 51m 11s
94:	learn: 0.3865750	test: 0.3822712	best: 0.3822712 (94)	total: 40m 7s	remaining: 2h 51m 3s
95:	learn: 0.3865583	test: 0.3822577	best: 0.3822577 (95)	total: 40m 31s	remaining: 2h 50m 33s
96:	learn: 0.3864879	test: 0.3821672	best: 0.3821672 (96)	total: 41m	remaining: 2h 50m 20s
97:	learn: 0.3864582	test: 0.3821104	best: 0.3821104 (97)	total: 41m 24s	remaining: 2h 49m 49s
98:	learn: 0.3864449	test: 0.3820880	best: 0.3820880 (98)	tot

173:	learn: 0.3853268	test: 0.3809103	best: 0.3809103 (173)	total: 1h 20m 19s	remaining: 2h 30m 28s
174:	learn: 0.3853174	test: 0.3809022	best: 0.3809022 (174)	total: 1h 20m 53s	remaining: 2h 30m 12s
175:	learn: 0.3853102	test: 0.3808949	best: 0.3808949 (175)	total: 1h 21m 17s	remaining: 2h 29m 39s
176:	learn: 0.3853043	test: 0.3808910	best: 0.3808910 (176)	total: 1h 21m 43s	remaining: 2h 29m 7s
177:	learn: 0.3853002	test: 0.3808875	best: 0.3808875 (177)	total: 1h 22m 10s	remaining: 2h 28m 39s
178:	learn: 0.3852908	test: 0.3808800	best: 0.3808800 (178)	total: 1h 22m 32s	remaining: 2h 28m 1s
179:	learn: 0.3852857	test: 0.3808760	best: 0.3808760 (179)	total: 1h 22m 54s	remaining: 2h 27m 23s
180:	learn: 0.3852675	test: 0.3808605	best: 0.3808605 (180)	total: 1h 23m 21s	remaining: 2h 26m 55s
181:	learn: 0.3852571	test: 0.3808536	best: 0.3808536 (181)	total: 1h 23m 46s	remaining: 2h 26m 23s
182:	learn: 0.3852509	test: 0.3808501	best: 0.3808501 (182)	total: 1h 24m 13s	remaining: 2h 25m 54s
18

256:	learn: 0.3845891	test: 0.3802210	best: 0.3802210 (256)	total: 1h 54m 34s	remaining: 1h 48m 19s
257:	learn: 0.3845821	test: 0.3802162	best: 0.3802162 (257)	total: 1h 55m	remaining: 1h 47m 52s
258:	learn: 0.3845783	test: 0.3802133	best: 0.3802133 (258)	total: 1h 55m 28s	remaining: 1h 47m 26s
259:	learn: 0.3845742	test: 0.3802117	best: 0.3802117 (259)	total: 1h 55m 51s	remaining: 1h 46m 56s
260:	learn: 0.3845681	test: 0.3802061	best: 0.3802061 (260)	total: 1h 56m 14s	remaining: 1h 46m 26s
261:	learn: 0.3845671	test: 0.3802054	best: 0.3802054 (261)	total: 1h 56m 38s	remaining: 1h 45m 57s
262:	learn: 0.3845637	test: 0.3802028	best: 0.3802028 (262)	total: 1h 57m 5s	remaining: 1h 45m 31s
263:	learn: 0.3845607	test: 0.3802027	best: 0.3802027 (263)	total: 1h 57m 27s	remaining: 1h 45m
264:	learn: 0.3845574	test: 0.3802011	best: 0.3802011 (264)	total: 1h 57m 50s	remaining: 1h 44m 30s
265:	learn: 0.3845549	test: 0.3801969	best: 0.3801969 (265)	total: 1h 58m 25s	remaining: 1h 44m 11s
266:	lear

339:	learn: 0.3841609	test: 0.3798483	best: 0.3798483 (339)	total: 2h 31m 15s	remaining: 1h 11m 10s
340:	learn: 0.3841600	test: 0.3798483	best: 0.3798483 (339)	total: 2h 31m 38s	remaining: 1h 10m 42s
341:	learn: 0.3841573	test: 0.3798478	best: 0.3798478 (341)	total: 2h 32m 4s	remaining: 1h 10m 15s
342:	learn: 0.3841552	test: 0.3798472	best: 0.3798472 (342)	total: 2h 32m 29s	remaining: 1h 9m 47s
343:	learn: 0.3841475	test: 0.3798350	best: 0.3798350 (343)	total: 2h 32m 54s	remaining: 1h 9m 20s
344:	learn: 0.3841315	test: 0.3798192	best: 0.3798192 (344)	total: 2h 33m 27s	remaining: 1h 8m 56s
345:	learn: 0.3841291	test: 0.3798188	best: 0.3798188 (345)	total: 2h 33m 50s	remaining: 1h 8m 28s
346:	learn: 0.3841257	test: 0.3798164	best: 0.3798164 (346)	total: 2h 34m 17s	remaining: 1h 8m 1s
347:	learn: 0.3841232	test: 0.3798165	best: 0.3798164 (346)	total: 2h 34m 42s	remaining: 1h 7m 34s
348:	learn: 0.3841223	test: 0.3798164	best: 0.3798164 (348)	total: 2h 35m 5s	remaining: 1h 7m 6s
349:	learn:

424:	learn: 0.3838240	test: 0.3795891	best: 0.3795891 (424)	total: 3h 11m 34s	remaining: 33m 48s
425:	learn: 0.3838222	test: 0.3795884	best: 0.3795884 (425)	total: 3h 12m 5s	remaining: 33m 22s
426:	learn: 0.3838146	test: 0.3795832	best: 0.3795832 (426)	total: 3h 12m 32s	remaining: 32m 55s
427:	learn: 0.3838030	test: 0.3795720	best: 0.3795720 (427)	total: 3h 13m 5s	remaining: 32m 28s
428:	learn: 0.3837980	test: 0.3795694	best: 0.3795694 (428)	total: 3h 13m 34s	remaining: 32m 2s
429:	learn: 0.3837952	test: 0.3795686	best: 0.3795686 (429)	total: 3h 14m 3s	remaining: 31m 35s
430:	learn: 0.3837927	test: 0.3795674	best: 0.3795674 (430)	total: 3h 14m 30s	remaining: 31m 8s
431:	learn: 0.3837884	test: 0.3795634	best: 0.3795634 (431)	total: 3h 15m	remaining: 30m 41s
432:	learn: 0.3837847	test: 0.3795595	best: 0.3795595 (432)	total: 3h 15m 27s	remaining: 30m 14s
433:	learn: 0.3837829	test: 0.3795599	best: 0.3795595 (432)	total: 3h 15m 53s	remaining: 29m 47s
434:	learn: 0.3837823	test: 0.3795595	b

# Bayesian Optimization 

In [ ]:
#testing out bayesian optimization which we did not ultimately use due to computation time
def report_perf(optimizer, X_train, y_train, title, callbacks=None):
    start = time()
    if callbacks:
        optimizer.fit(X_train, y_train, callback=callbacks)
    else:
        optimizer.fit(X_train, y_train)
    d=pd.DataFrame(optimizer.cv_results_)
    best_score = optimizer.best_score_
    best_score_std = d.iloc[optimizer.best_index_].std_test_score
    best_params = optimizer.best_params_
    print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
           +u"\u00B1"+" %.3f") % (time() - start, 
                                  len(optimizer.cv_results_['params']),
                                  best_score,
                                  best_score_std))    
    print('Best parameters:')
    pprint.pprint(best_params)
    print()
    return best_params

clf = cb.CatBoostClassifier(
    loss_function='Logloss',
    custom_metric='Logloss',
    random_seed=19,
    verbose=1,
    thread_count=-1,
    od_type='Iter',
    od_wait=20,
    iterations=1000,
    task_type='GPU',
    cat_features=cat_features)

search_spaces = {'learning_rate': Real(0.001, 0.05, 'log-uniform'),
                'depth': Int(6,10),
                'iterations': Int(500,1000)}

outer_cv = KFold(n_splits=5, shuffle=True,random_state = 1)

opt = BayesSearchCV(clf,
                    search_spaces,
                    scoring='neg_log_loss',
                    cv=outer_cv,
                    n_iter=10,
                    n_jobs=-1,  
                    return_train_score=False,
                    refit=True,
                    random_state=42,
                   verbose=1)

best_params = report_perf(opt, X_train, y_train,'CatBoost')

# Model evaluation

In [11]:
#validation set log loss score
catboost_prob = catboost.predict_proba(val_x)
catboost_log_loss = log_loss(val_y, catboost_prob)
catboost_log_loss

0.37998205530266455

In [41]:
#feature importance
df_feature_importance = pd.DataFrame(catmodel.get_feature_importance(prettified=True))
df_feature_importance

,Feature Id,Importances
0,0,21.381188
1,3,20.828027
2,8,9.603276
3,7,8.920312
4,11,7.396351
5,14,5.820986
6,1,4.425710
7,6,4.276997
8,19,2.518632
9,18,2.423634


# Predicting probabilities for the test set

In [11]:
#Downloading the test data file and cleaning the data to feed into the model
test_data=pd.read_csv(r"C:\Users\klee9\Downloads\ProjectTestData.csv")

test_data['hr'] = test_data['hour'].astype(str).str[-2:]
test_data['day'] = test_data['hour'].astype(str).str[4:6]
test_data['C1'] = test_data['C1'].astype(str)
test_data['banner_pos'] = test_data['banner_pos'].astype(str)
test_data['device_type'] = test_data['device_type'].astype(str)
test_data['device_conn_type'] = test_data['device_conn_type'].astype(str)
test_data['C14'] = test_data['C14'].astype(str)
test_data['C15'] = test_data['C15'].astype(str)
test_data['C16'] = test_data['C16'].astype(str)
test_data['C17'] = test_data['C17'].astype(str)
test_data['C18'] = test_data['C18'].astype(str)
test_data['C19'] = test_data['C19'].astype(str)
test_data['C20'] = test_data['C20'].astype(str)
test_data['C21'] = test_data['C21'].astype(str)

test_data = test_data.drop('hour', 1)
test_data.loc[test_data['site_id'].value_counts()[test_data['site_id']].values < (.000001*test_data.shape[0]), 'site_id'] = "Other"
test_data.loc[test_data['site_domain'].value_counts()[test_data['site_domain']].values < (.000001*test_data.shape[0]), 'site_domain'] = "Other"
test_data.loc[test_data['app_id'].value_counts()[test_data['app_id']].values < (.000001*test_data.shape[0]), 'app_id'] = "Other"
test_data.loc[test_data['app_domain'].value_counts()[test_data['app_domain']].values < (.000001*test_data.shape[0]), 'app_domain'] = "Other"
test_data.loc[test_data['device_id'].value_counts()[test_data['device_id']].values < (.000001*test_data.shape[0]), 'device_id'] = "Other"
test_data.loc[test_data['device_ip'].value_counts()[test_data['device_ip']].values < (.000001*test_data.shape[0]), 'device_ip'] = "Other"
test_data.loc[test_data['device_model'].value_counts()[test_data['device_model']].values < (.000001*test_data.shape[0]), 'device_model'] = "Other"
test_data.loc[test_data['C14'].value_counts()[test_data['C14']].values < (.000001*test_data.shape[0]), 'C14'] = "Other"
test_data.loc[test_data['C17'].value_counts()[test_data['C17']].values < (.000001*test_data.shape[0]), 'C17'] = "Other"
test_data.loc[test_data['C20'].value_counts()[test_data['C20']].values < (.000001*test_data.shape[0]), 'C20'] = "Other"

test_data=test_data.drop(["id"], axis=1)


test_data=test_data.values
test_data=test_data[:,2:]

test_prob = catboost.predict_proba(test_data)
df_preds = pd.DataFrame(test_prob[:,1], columns = ['P(click)'])

In [35]:
#merging the predicted probailities with the submission file and saving as csv
submission_data=pd.read_csv(r"C:\Users\klee9\Downloads\ProjectSubmission-TeamX.csv")
submission_data.drop(submission_data.columns[1], axis=1, inplace=True)
concatenated = pd.concat([submission_data, df_preds], axis=1)
concatenated.to_csv(r"C:\Users\klee9\Downloads\ProjectSubmission-Team8.csv", index=False)